In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/a171e61840.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/4171ea3a0c.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/0f985a8440.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/5d320ade20.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/ae8c67086b.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/b7214cbf21.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/e18fcafee8.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/79568b8e25.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/feba449e1a.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/7ebad45aec.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/d9bf33ecd9.csv
/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdc